In [0]:
# dbutils.widgets.text("src","")

In [0]:
# src_value = dbutils.widgets.get("src")
# src_value

In [0]:
# # Create a streaming DataFrame using Databricks Auto Loader (cloudFiles)
# df = (
#     spark.readStream.format("cloudFiles")
#     # Specify the format of incoming files (CSV in this case)
#     .option("cloudFiles.format", "csv")
#     # Location to store schema and checkpoint info
#     # - Tracks which files are already processed (avoids duplicates)
#     # - Stores inferred schema so it's not recomputed each time
#     .option("cloudFiles.schemaLocation", f"/Volumes/workspace/flight_bronze/bronzevolume/{src_value}/checkpoint")
#     # Handle schema evolution:
#     # "rescue" means unexpected new columns are captured in "_rescued_data" instead of failing
#     .option("cloudFiles.schemaEvolutionMode", "rescue")
#     # Path where Auto Loader will look for new incoming CSV files
#     # Uses src_value variable so each source has its own folder
#     .load(f"/Volumes/workspace/flight_raw/rawvolume/rawdata/{src_value}/")
# )


In [0]:
# # Write the streaming DataFrame into a Delta table (Bronze layer)
# (
#     df.writeStream.format("delta")
#     # Append mode: new data gets added without modifying existing records
#     .outputMode("append")
#     # Trigger set to "once":
#     .trigger(once=True)
#     # Checkpoint location for fault tolerance
#     .option("checkpointLocation", f"/Volumes/workspace/flight_bronze/bronzevolume/{src_value}/checkpoint")
#     # Final storage path for the Delta table
#     .option("path", f"/Volumes/workspace/flight_bronze/bronzevolume/{src_value}/data")
#     # Start the streaming write query
#     .start()
# )


In [0]:
# %sql
# SELECT * FROM delta.`/Volumes/workspace/flight_bronze/bronzevolume/customers/data`

In [0]:
# %sql
# -- Make sure schema exists
# CREATE SCHEMA IF NOT EXISTS workspace.flight_raw;
# CREATE SCHEMA IF NOT EXISTS workspace.flight_bronze;

# -- Register the Volumes (this tells UC about the folders you already have)
# CREATE VOLUME IF NOT EXISTS workspace.flight_raw.raw_volume;
# CREATE VOLUME IF NOT EXISTS workspace.flight_bronze.bronzevolume;


In [0]:
# %sql
# SHOW VOLUMES IN workspace.flight_raw;
# SHOW VOLUMES IN workspace.flight_bronze;


In [0]:

import yaml


yaml_path = "/Workspace/Users/tushhaar24@gmail.com/Databricks_flight_project/Databricks_Flight_Data_Project/configs/autoloader_bronze.yml"

with open(yaml_path, "r") as f:
    config = yaml.safe_load(f)

autoloader_cfg = config["autoloader"]


file_format = autoloader_cfg["file_format"]
options = autoloader_cfg.get("options", {})
checkpoint_template = autoloader_cfg["checkpoint_location"]
target_template = autoloader_cfg["target_path"]
source_template = autoloader_cfg["source_path"]
tables = autoloader_cfg["tables"]


dbutils.widgets.text("src", "")
src_value = dbutils.widgets.get("src")


if src_value and src_value in tables:
    tables_to_process = [src_value]
else:
    tables_to_process = tables

print(f"Tables selected for ingestion: {tables_to_process}")


for table in tables_to_process:
    print(f"Processing table: {table}")

    
    checkpoint_path = checkpoint_template.format(table=table)
    target_path = target_template.format(table=table)
    source_path = source_template.format(table=table)

    
    df = (
        spark.readStream.format("cloudFiles")
        .option("cloudFiles.format", file_format)
        .options(**options)  
        .option("cloudFiles.schemaLocation", checkpoint_path)
        .load(source_path)
    )

    
    (
        df.writeStream.format("delta")
        .outputMode("append")
        .trigger(once=True)
        .option("checkpointLocation", checkpoint_path)
        .option("path", target_path)
        .start()
    )

print("Bronze ingestion job submitted.")
